In [ ]:
import wandb, os, sys

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Running in Colab!")
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    from google.colab import userdata
    WANDB_KEY = userdata.get('WANDB_KEY')
    wandb.login(key=WANDB_KEY)
else:
    print("Not running in Colab.")

def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/nn_catalyst/' + relativePath
    else:
        from utils import get_project_root
        return get_project_root() + "/" + relativePath

wandb.init(project="nn_catalyst")

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW

# Load the data
descriptors_path = 'descriptors.csv'
targets_path = 'compiled_data.csv'

In [ ]:
descriptors_df = pd.read_csv(resolve_path_gdrive(descriptors_path))
targets_df = pd.read_csv(resolve_path_gdrive(targets_path))

In [ ]:
# Show sample rows
print("\nSample Rows from Descriptors DataFrame:")
print(descriptors_df.head())
print("\nSample Rows from Targets DataFrame:")
print(targets_df.head())

In [ ]:
# selected column
selected_cols=[5, 14, 15, 24, 25]
number_of_target_cols = len(selected_cols)
selected_cols.insert(0, 0)
targets_df = targets_df.iloc[:, selected_cols]
print(targets_df)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
wandb.log({"target cols": selected_cols})

In [20]:
# Keep only numeric columns
descriptors_numeric = descriptors_df.select_dtypes(include=['number'])
targets_numeric = targets_df.select_dtypes(include=['number'])

# Merge the numeric dataframes on the common label column
numeric_data = pd.merge(descriptors_numeric, targets_numeric, left_on='Label', right_on='mol_num')
numeric_data = numeric_data.drop(columns=['Label', 'mol_num'])

# Separate features and targets
X = numeric_data.iloc[:, :-number_of_target_cols]  # Assuming the last 30 columns are targets
y = numeric_data.iloc[:, -number_of_target_cols:]

In [21]:
# Apply variance threshold
selector = VarianceThreshold()
X_high_variance = selector.fit_transform(X)

# Convert to numpy arrays
X = X_high_variance
y = y.values

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the data
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

X_train = scaler_X.transform(X_train)
X_val = scaler_X.transform(X_val)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.transform(y_train)
y_val = scaler_y.transform(y_val)
y_test = scaler_y.transform(y_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32, device=device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32, device=device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

In [22]:
# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the individual model class
class SingleTargetNet(nn.Module):
    def __init__(self, input_size, dropout_rate=0.5):
        super(SingleTargetNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 1)
        self.fc_skip = nn.Linear(1024, 512)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x1 = F.relu(self.bn1(self.fc1(x)))
        x1 = self.dropout(x1)

        x2 = F.relu(self.bn2(self.fc2(x1)))
        x2 = self.dropout(x2)

        # Skip connection
        x2 += self.fc_skip(x1)

        x3 = self.fc3(x2)
        return x3

def get_target5model():
    # Define the model
    model = nn.Sequential(
        nn.Linear(X_train.shape[1], 1024),
        nn.LeakyReLU(),
        nn.Linear(1024, 512),
        nn.LeakyReLU(),
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        nn.Linear(256, 1)
    )
    return model

class RegressionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RegressionNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [23]:
# Function to train and evaluate individual models
def train_and_evaluate(target_index, model):
    model.to(device=device)
    criterion = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

    best_val_loss = np.inf
    patience_counter = 0
    num_epochs = 150

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Target {target_index} - Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, targets[:, target_index])
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f'Target {target_index} - Validation Loss: {val_loss}')

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f'saved_models/{selected_cols[target_index+1]}_{type(model).__name__}_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= 15:
                print(f'Target {target_index} - Early stopping triggered')
                break

    model.load_state_dict(torch.load(f'saved_models/{selected_cols[target_index+1]}_{type(model).__name__}_model.pth'))

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets[:, target_index])
            test_loss += loss.item()
    test_loss /= len(test_loader)
    print(f'Target {target_index} - Test Loss: {test_loss}')
    
    return model, test_loss

In [24]:
# Train and evaluate individual models for each target
test_losses = []
models = []
try_models = [SingleTargetNet(X_train.shape[1]), RegressionNetwork(X_train.shape[1], 512, 1), get_target5model()]

for target_index in range(y_train.shape[1]):
    for a_model in try_models:
        model, test_loss = train_and_evaluate(target_index, a_model)
        models.append(model)
        test_losses.append(test_loss)

SyntaxError: incomplete input (3501983771.py, line 10)

In [25]:
# Prepare DataFrames for train, validation, and test predictions
train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()

r2_scores, rmse_scores, mae_scores = [], [], []

def compute_stats(target_index, df):
    print(df)
    observed_col = f'Observed_{target_index}'
    predicted_col = f'Predicted_{target_index}'

    # Calculate metrics
    observed = df[observed_col]
    predicted = df[predicted_col]
    r2 = r2_score(observed, predicted)
    rmse = mean_squared_error(observed, predicted, squared=False)
    mae = mean_absolute_error(observed, predicted)
    return r2, rmse, mae

In [26]:
def evaluate(target_index, model):
    # Make predictions on the train, validation, and test sets
    model.cpu().eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor.cpu()).numpy()
        y_val_pred = model(X_val_tensor.cpu()).numpy()
        y_test_pred = model(X_test_tensor.cpu()).numpy()

    # Inverse transform the predictions and targets to their original scale
    y_train_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_train_pred.shape[0], target_index)), y_train_pred, np.zeros((y_train_pred.shape[0], y_train.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_val_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_val_pred.shape[0], target_index)), y_val_pred, np.zeros((y_val_pred.shape[0], y_val.shape[1] - target_index - 1))], axis=1))[:, target_index]
    y_test_pred_orig = scaler_y.inverse_transform(np.concatenate([np.zeros((y_test_pred.shape[0], target_index)), y_test_pred, np.zeros((y_test_pred.shape[0], y_test.shape[1] - target_index - 1))], axis=1))[:, target_index]

    y_train_orig = scaler_y.inverse_transform(y_train)[:, target_index]
    y_val_orig = scaler_y.inverse_transform(y_val)[:, target_index]
    y_test_orig = scaler_y.inverse_transform(y_test)[:, target_index]

    # Create dataframes for the predictions and actual values
    train_df[f'Observed_{target_index}'] = y_train_orig
    train_df[f'Predicted_{target_index}'] = y_train_pred_orig

    val_df[f'Observed_{target_index}'] = y_val_orig
    val_df[f'Predicted_{target_index}'] = y_val_pred_orig

    test_df[f'Observed_{target_index}'] = y_test_orig
    test_df[f'Predicted_{target_index}'] = y_test_pred_orig

    # Create and insert parity plots for train, validation, and test sets
    r2, rmse, mae = compute_stats(target_index, train_df)
    r2_scores.append(r2)
    rmse_scores.append(rmse)
    mae_scores.append(mae)
    
for target_index in range(y_train.shape[1]):
    for model_type in try_models:
        model = model_type
        model.load_state_dict(torch.load(f'saved_models/{selected_cols[target_index+1]}_{type(model_type).__name__}_model.pth'))
        evaluate(target_index, model)


C:\Users\dalje\AppData\Local\Temp\ipykernel_20604\1490988916.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'saved_models/{selected_c

       Observed_0  Predicted_0
0         3.03390     3.133207
1         2.21883     4.118872
2         3.42129     1.926924
3         6.54626     5.892724
4         2.70455     1.972907
...           ...          ...
20981     5.82050     5.541566
20982     2.90117     1.944907
20983     3.06542     2.879624
20984     2.26019     2.265115
20985     1.91199     2.678473

[20986 rows x 2 columns]


d:\work\conda\nn_310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\dalje\AppData\Local\Temp\ipykernel_20604\1490988916.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recom

       Observed_0  Predicted_0
0         3.03390     3.439919
1         2.21883     4.159586
2         3.42129     1.921974
3         6.54626     5.122129
4         2.70455     2.287310
...           ...          ...
20981     5.82050     5.667760
20982     2.90117     2.209077
20983     3.06542     3.142845
20984     2.26019     2.896136
20985     1.91199     2.429804

[20986 rows x 2 columns]


d:\work\conda\nn_310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\dalje\AppData\Local\Temp\ipykernel_20604\1490988916.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recom

...

d:\work\conda\nn_310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\dalje\AppData\Local\Temp\ipykernel_20604\1490988916.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recom

       Observed_0  Predicted_0  Observed_1  Predicted_1  Observed_2  \
0         3.03390     3.824085    0.498762     0.484318     3.89562   
1         2.21883     4.577974    0.162431     0.161983     7.08420   
2         3.42129     2.198584    0.547589     0.507206     6.47881   
3         6.54626     5.651527    0.308542     0.325894     5.39737   
4         2.70455     1.726750    0.572807     0.594937    11.06172   
...           ...          ...         ...          ...         ...   
20981     5.82050     5.444175    0.342817     0.365858     6.03192   
20982     2.90117     2.319813    0.485546     0.463759     3.37806   
20983     3.06542     3.316987    0.210382     0.199105    11.38933   
20984     2.26019     2.264390    0.336572     0.321267     1.31823   
20985     1.91199     2.267551    0.211747     0.204855     2.01098   

       Predicted_2  Observed_3  Predicted_3  Observed_4  Predicted_4  
0         3.822650    0.539819     0.549443    11.33621     9.425328  
1    

d:\work\conda\nn_310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\dalje\AppData\Local\Temp\ipykernel_20604\1490988916.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recom

       Observed_0  Predicted_0  Observed_1  Predicted_1  Observed_2  \
0         3.03390     3.824085    0.498762     0.484318     3.89562   
1         2.21883     4.577974    0.162431     0.161983     7.08420   
2         3.42129     2.198584    0.547589     0.507206     6.47881   
3         6.54626     5.651527    0.308542     0.325894     5.39737   
4         2.70455     1.726750    0.572807     0.594937    11.06172   
...           ...          ...         ...          ...         ...   
20981     5.82050     5.444175    0.342817     0.365858     6.03192   
20982     2.90117     2.319813    0.485546     0.463759     3.37806   
20983     3.06542     3.316987    0.210382     0.199105    11.38933   
20984     2.26019     2.264390    0.336572     0.321267     1.31823   
20985     1.91199     2.267551    0.211747     0.204855     2.01098   

       Predicted_2  Observed_3  Predicted_3  Observed_4  Predicted_4  
0         3.822650    0.539819     0.549443    11.33621    10.534435  
1    

d:\work\conda\nn_310\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
r2_scores, rmse_scores, mae_scores